In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep

In [5]:
options = Options()
# options.add_argument("--headless")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options = options)

pages = 1
while pages <= 2 :
    base_URL = "https://www.amiami.com/eng/search/list/?s_condition_flg=1&s_st_condition_flg=1&s_sortkey=preowned&pagecnt="
    final_URL = base_URL + str(pages)
    driver.get(final_URL)
    print(final_URL)

    page_html = driver.page_source
    soup = BeautifulSoup(page_html, 'html.parser')
    print(soup)
    # productList = soup.find_all('section', class_="item-about")
    # for product in productList:
    #     productData = product.find('dd')

    #     print(productData)
    #     print(type(productData))
    break
    pages += 1

        

https://www.amiami.com/eng/search/list/?s_condition_flg=1&s_st_condition_flg=1&s_sortkey=preowned&pagecnt=1
<html class="no-js" lang="en-US"><!--<![endif]--><head>
<title>Attention Required! | Cloudflare</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function () {
      var cookieEl = document.getElementById('cookie-alert');
      cookieEl.style.display = 'block';
    })
  }
</script>
<!--<![endif]-->
</head>
<body

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep
import time

# Setup Selenium options
options = Options()
# Do NOT use headless mode initially to see the CAPTCHA/challenge
# options.add_argument("--headless")  # You can enable this later
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize the driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Open the website and wait for it to load (to bypass any JavaScript challenges)
base_URL = "https://www.amiami.com/eng/search/list/?s_condition_flg=1&s_st_condition_flg=1&s_sortkey=preowned&pagecnt="
pages = 1
final_URL = base_URL + str(pages)
driver.get(final_URL)
# driver.get("https://www.amiami.com/eng/search/list/?s_keywords=pre-owned+figures")
time.sleep(10)  # Wait for Cloudflare's challenge to pass if any

# After the challenge is passed, you can scrape the page content
page_html = driver.page_source
# print(page_html)
soup = BeautifulSoup(page_html, 'html.parser')
print(soup)
productList = soup.find_all('section', class_="item-about")

for product in productList:
    productData = product.find('dd')

    print(productData)
    print(type(productData))


driver.quit()

NameError: name 'BeautifulSoup' is not defined